# Prepare corpora for SAGE
Then run it at ~/SAGE/py-sage (repo fork at https://github.com/michaelmilleryoder/SAGE/tree/master/py-sage)  
Take a look at runSage_grid.py, which runs runSage.py over multiple parameter settings

In [1]:
import json
import os
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm
from multiprocessing import Pool

In [10]:
def process_tweet_dump(section):
    """ Extract text and tokenize a set of tweet dumps """
    
    texts = []
#     dirpath = os.path.join('/home/huixiann/2022_socialbias_vaccine/', section)    
    dirpath = os.path.join('/home/huixiann/2022_socialbias_vaccine/1_bycontinent', section)
    for fname in tqdm(os.listdir(dirpath)):
        if fname.startswith('.'):
            continue
        fpath = os.path.join(dirpath, fname)
        with open(fpath) as f:
            lines = f.read().splitlines()
            for line in lines:
                try:
                    tweet = json.loads(line)
                except ValueError:
                    continue
                text = ' '.join(tokenizer.tokenize(tweet['text'])).lower()
                texts.append(text)

    # Save out
    out_dirpath = os.path.join('../input/1_bycontinent')
    if not os.path.exists(out_dirpath):
        os.mkdir(out_dirpath)
    outpath = os.path.join(out_dirpath, '{}_sents.txt'.format(section))
    with open(outpath, 'w') as f:
        for sent in texts:
            f.write(sent.encode('utf8') + '\n')

In [11]:
# Explore json tweets Lynnette files
tokenizer = TweetTokenizer(strip_handles=True)
# pool = Pool(4)
# sections = ['0_anti-bot', '0_anti-human', '0_pro-bot', '0_pro-human']
pool = Pool(8)
# sections = sorted(os.listdir('/home/huixiann/2022_socialbias_vaccine/1_bycontinent/'))
sections = ['others']

# list(tqdm(pool.imap(process_tweet_dump, sections), total=len(sections)))
list(map(process_tweet_dump, sections)) # for debugging

100%|██████████| 56/56 [00:48<00:00,  1.15it/s]


[None]

In [ ]:
# for section in ['0_anti-bot', '0_anti-human', '0_pro-bot', '0_pro-human']:
#     print(section)
#     texts = []
#     dirpath = os.path.join('/home/huixiann/2022_socialbias_vaccine/', section)
#     for fname in tqdm(os.listdir(dirpath)):
#         if fname.startswith('.'):
#             continue
#         fpath = os.path.join(dirpath, fname)
#         with open(fpath) as f:
#             lines = f.read().splitlines()
#             for line in lines:
#                 tweet = json.loads(line)
#                 text = ' '.join(tokenizer.tokenize(tweet['text'])).lower()
#                 texts.append(text)

#     # Save out
#     out_dirpath = os.path.join('../input')
#     if not os.path.exists(out_dirpath):
#         os.mkdir(out_dirpath)
#     outpath = os.path.join(out_dirpath, '{}_sents.txt'.format(section))
#     with open(outpath, 'w') as f:
#         for sent in texts:
#             f.write(sent.encode('utf8') + '\n')